In [63]:
import kiutils.symbol
import os
import glob
import pandas as pd
import numpy as np
class KicadDB:
    def __init__(self, kicad_symbols_path, database_parts_path):
        self.kicad_parts_df, self.vendors_df = load_kicad_lib_as_dataframe(kicad_symbols_path)
        self.database_parts_df = pd.read_excel(database_parts_path).fillna('')


    def check_kicad_for_duplicate_BRIDs(self):

        parts_df = self.kicad_parts_df.copy()
        
        # This returns a Series of booleans indicating whether a given entry is a duplicate or not
        # BR ID is just a column for the Kicad library
        dupes = parts_df.duplicated(subset=['BR ID'], keep=False)

        # Multiple NAs are not duplicates...
        offenders = parts_df[dupes]
        offenders = offenders[offenders['BR ID'] != 'NA']

        # Raise an assertion error if there are duplicates in the Kicad symbol library
        assert len(offenders) == 0,  f"The following parts have the same BR IDs in the BR Kicad Symbol Library. One of em aint right. YOU MUST RESOLVE THIS ERROR GODDAMNIT.\n {offenders}"
        print("No BR ID duplicates found in Kicad library!")

    def check_database_for_duplicate_BRIDs(self):

        parts_df = self.database_parts_df.copy()

        # BR ID is now just a column in the sheet when processing
        dupes = parts_df.duplicated(subset=['BR ID'], keep=False)
        # Multiple NAs are not duplicates...
        offenders = parts_df[dupes]
        offenders = offenders[offenders['BR ID'] != 'NA']

        # Raise an assertion error if there are duplicates in the spreadsheet
        assert len(offenders) == 0,  f"The following parts have the same BR IDs in the spreadsheet. One of em aint right. YOU MUST RESOLVE THIS ERROR GODDAMNIT.\n {offenders}"
        print("No BR ID duplicates found in sheet!")
    """
    def check_sheet_for_duplicate_BRIDs(parts_df):

        # This returns a Series of booleans indicating whether a given entry is a duplicate or not
        # BR ID is the index of the sheet

        dupes = parts_df.index.duplicated(keep=False)
        offenders = parts_df[dupes]

        # Raise an assertion error if there are duplicates in the spreadsheet
        assert len(offenders) == 0,  f"The following parts have the same BR IDs in the spreadsheet. One of em aint right. YOU MUST RESOLVE THIS ERROR GODDAMNIT.\n {offenders}"
        print("No BR ID duplicates found in sheet!")
    """

    def add_new_symbols_from_kicad(self):

        # Do process on copies of the data and then reassign to the class attributes
        kicad_parts_df = self.kicad_parts_df.copy()
        database_parts_df = self.database_parts_df.copy()

        new_parts = kicad_parts_df[kicad_parts_df["BR ID"]=='NA']
        if len(new_parts) > 0:
            while(1):
                user_confirmation = input(f"New parts detected: \n{new_parts['Name'].to_list()}\n Would you like to assign these to BR IDs and add to database?\n [y/n]")
                if user_confirmation == 'y':
                    for i in new_parts.index.to_list():
                        # Scan the both Kicad and database for existing BRIDs so we don't accidentally make multiple parts with the same BRID
                        existing_BRIDs = list_union(kicad_parts_df['BR ID'].to_list(), database_parts_df['BR ID'].to_list())
                        new_BRID = generate_BRID(existing_BRIDs)


                    
                        kicad_parts_df.loc[i, "BR ID"] = new_BRID

                        print(f"{kicad_parts_df.loc[i, 'Name']} assigned to {new_BRID}")

                        # Add part to database
                        database_parts_df = pd.concat([database_parts_df, kicad_parts_df.loc[[i]]], ignore_index=True, axis=0)
                        print(f"New Kicad part {new_BRID} added to database")

                    break

                elif user_confirmation == 'n':
                    print("Part not added, you need to add a unique BR ID in order to use the database")
                    break
                
                else:
                    print("invalid response")
        else:
            print("No new parts detected from Kicad library")
        
        self.kicad_parts_df = kicad_parts_df.copy()
        self.database_parts_df = database_parts_df.copy()

    def deal_with_extra_parts(self):
        
        # Comparison and updates
        kicad_parts_df = self.kicad_parts_df.set_index("BR ID")
        kicad_parts_df.sort_index(inplace=True)
        database_parts_df = self.database_parts_df.set_index("BR ID").sort_index()
        database_parts_df.sort_index(inplace=True)

        # Figure out if there are any outlier parts that are only accounted for in one place
        kicad_extra_BRIDs = kicad_parts_df.index.difference(database_parts_df.index).to_list()
        database_extra_BRIDs = database_parts_df.index.difference(kicad_parts_df.index).to_list()

        # If there are unaccounted for parts, we need to select how we deal with them
        if len(kicad_extra_BRIDs) + len(database_extra_BRIDs) > 0: 

            # Request input from the user as to how the merge should take place. Options are:
            # 1. Merge all new parts Kicad <--> Database
            # 2. Only merge Kicad --> Database (delete extras from database)
            # 3. Only merge Database --> Kicad (delete extras from Kicad)
            # 4. Delete all outlier components
            # in the future, add a fifth option that shows more information about the parts in question (or maybe a more custom option?)
            merge_option = 0 # VScode's being annoying if I don't initialize it outside of the loop
            while(1):
                merge_option = input(f"There are unaccounted parts between the library and database \n Only in Kicad Library: {kicad_extra_BRIDs} \n Only in the Database: {database_extra_BRIDs} \n You have four options [1-4]: \n [1] Merge all new parts in both directions \n [2] Push only Kicad --> Database (delete any extras in db) \n [3] Push only Database -- Kicad (delete any extras in Kicad) \n [4] Delete all extra parts that are unaccounted for")
                merge_option = int(merge_option)
                if merge_option in [1, 2, 3, 4]:
                    break
                else:
                    print("Invalid input (must be 1-4)")
            
            # Extra BR IDs in the Kicad library that are unaccounted for in the database need to be managed (merged or deleted)
            if len(kicad_extra_BRIDs) > 0:

                # If 1 or 2 was selected, add these extra kicad parts to the database
                if merge_option in [1, 2]: 
                    print(kicad_parts_df.loc[[kicad_extra_BRIDs]])
                    database_parts_df = pd.concat([database_parts_df, kicad_parts_df.loc[[kicad_extra_BRIDs]]], sort=True, axis=0)

                # Otherwise, delete these extra parts from the Kicad library
                else:
                    kicad_parts_df.drop(kicad_extra_BRIDs, inplace=True)


            # Extra BR IDs in the database that are unaccounted for in the Kicad library need to managed (merged or deleted)
            if len(database_extra_BRIDs) > 0:

                # If 1 or 2 was selected, add these extra database parts to the Kicad library
                if merge_option in [1, 3]: 
                    kicad_parts_df = pd.concat([kicad_parts_df, database_parts_df.loc[[database_extra_BRIDs]]], sort=True, axis=0)

                # Otherwise, delete these extra parts from the database
                else:
                    database_parts_df.drop(database_extra_BRIDs, inplace=True)

        # If there are no unaccounted for parts, we can just do a clean update
        else: 

            print("All parts are accounted for, check for updates")
        
        self.kicad_parts_df = kicad_parts_df.copy()
        self.database_parts_df = database_parts_df.copy()

        



def load_kicad_lib_as_dataframe(symbols_path):
    
    os.chdir(symbols_path)

    parts_list = []
    vendors_list = []
    
    print("Loading in libraries from Kicad...")
    
    for lib_file in glob.glob("*.kicad_sym"):

        # Extract library nickname/category -- e.g., 0402_Capacitors
        lib_nickname = lib_file.replace(".kicad_sym", "")

        # Skip these libraries, they don't need to be documented (obsolete or not actual parts)
        if (lib_nickname == "BR~Deprecated") or (lib_nickname == "BR_Virtual_Parts"):
            continue

        # Open symbol library
        lib_path = os.path.join(symbols_path, lib_file)
        symbol_lib = kiutils.symbol.SymbolLib().from_file(lib_path)
        
        # The Category is the library nickname, without the BR_ at the beginning
        category = lib_nickname[3:]



        # For each symbol in a given library, populate a new row in the Parts dataframe
        for symbol in symbol_lib.symbols:

            # Symbol path in Kicad
            symbol_path = f"{lib_nickname}:{symbol.entryName}"

            # Grab all the properties from the Kicad Symbol
            properties = {property.key: property.value for property in symbol.properties}

            # Some parts don't have a manufacturer and manufacturer part number -- deal with this some other time, for now just populate "None"
            if "Manufacturer" in properties:
                manufacturer = properties["Manufacturer"]
                mpn = properties["Manufacturer Part Num"]
            else:
                manufacturer = ""
                mpn = ""

            if "BR ID" in properties:
                BR_ID = properties["BR ID"]
                if len(BR_ID) < 7:
                    BR_ID = "NA"
            else:
                BR_ID = "NA"
            # Append a dictionary of all part properties to the parts list -- this will be converted to a Pandas dataframe at the end
            parts_list.append({"BR ID":BR_ID, "Name":symbol.libId, "Description":properties["Description"], "Value":properties["Value"], "Symbol":symbol_path, "Footprint":properties["Footprint"],  "Datasheet":properties["Datasheet"], "Manufacturer":manufacturer, "MPN":mpn, "Category":category})

            # Extract all supplier-related properties: supplier X with suppler number X
            supplier_properties = {property: properties[property] for property in properties if property[:8]=="Supplier"}
            supplier_numbers = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]=='P'}
            supplier_names = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]!='P'}

            # Ignore anything that looks like this
            null_strings = ["", " ", "-", "--", "~", "NA", "N/A"]

            # Loop through and add vendors and the respective supplier number when the number X at the end matches (supplier 1 --> supplier part num 1)
            for name in supplier_names:
                for number in supplier_numbers:
                    if name[-1] == number[-1]:
                        if supplier_numbers[number] not in null_strings:

                            # Append dictionary of supplier properties for each SPN to the vendors list
                            # this will be converted to Pandas dataframe at the end
                            vendors_list.append({"BR ID":BR_ID, "Supplier":supplier_names[name], "SPN":supplier_numbers[number], "Stock":0})



    # Create Pandas dataframes from these lists of dictionaries
    # Think of each dictionary as a row in the table
    parts_df = pd.DataFrame(parts_list)
    vendors_df = pd.DataFrame(vendors_list)
    
    return parts_df, vendors_df

def list_union(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    set_union = set1.union(set2)
    return list(set_union)

#### Function for generating a unique BR ID ####
def generate_BRID(existing_BRIDs):

    # Set number of digits in unique BRID
    num_digits = 6      

    biggest_BRID = max([int(ID[-6:]) for ID in existing_BRIDs if len(ID)>6])

    # Increment by 1 until the BRID doesn't exist
    for n in range(biggest_BRID + 1, 10**num_digits):

        # Convert to a zero padded string (to fill the desired number of digits -- e.g, 1 --> '0001')
        BR_num = str(n).zfill(num_digits)
        BRID = 'BRE-' + BR_num

        # If the new BRID isn't currently used, we've found the one!
        if BRID not in existing_BRIDs:
            break

    return BRID


    

            

In [64]:



"""
loop through Kicad symbol libraries
For each Kicad symbol, check the parts sheet for a matching BR number
    If the BR number exists, check for changes
        If changed, make edits
        Otherwise, don't touch that part
    If the BR number doesn't exist, raise an error. All BR numbers should originate in the parts sheet
    If there is no BR ID field for a part, generate a new BR ID and fill out sheet accordingly using the Kicad symbol fields
            "ID":BR_ID, "Name":symbol.libId, "Description":properties["Description"], "Value":properties["Value"], "Symbol":symbol_path, "Footprint":properties["Footprint"],  "Datasheet":properties["Datasheet"], "Manufacturer":manufacturer, "MPN":mpn, "Category":category
    At the end, create a new row in the vendor sheet

Functions to add:
check_for_duplicate_BRID()
check_for_changes()
update_part_in_sheet()
add_BRID_to_kicad_part()
add_part_to_vendor_sheet()
            
"""
# Not really necessary, but initializes the dataframes. This is at good reference for the columns, at least
parts_df = pd.DataFrame(columns=['BR ID','Name','Description','Value','Symbol','Footprint','Datasheet','Manufacturer','MPN', 'Category'])
vendors_df = pd.DataFrame(columns=['BR ID','Supplier','SPN','Stock'])



# Reading symbol libraries from our BR symbols folder
# SYMBOLS_PATH = "C:/Users/JacobBrotmanKrass/Documents/GitHub/br-kicad-lib/Symbols"
# JLC_PATH = r"C:/Users/JacobBrotmanKrass/Documents/GitHub/br-components-database/jlc-scraper/csv/Parts Inventory on JLCPCB.xlsx"
# PARTS_SHEET_PATH = "C:/Users/JacobBrotmanKrass/Documents/GitHub/br-components-database/Kicad/Parts_Library.xlsx"
SYMBOLS_PATH = r"C:/Users/JacobBrotmanKrass/Documents/Test Library/Symbols"
JLC_PATH = r"C:/Users/JacobBrotmanKrass/Documents/GitHub/br-components-database/jlc-scraper/csv/Parts Inventory on JLCPCB.xlsx"
PARTS_SHEET_PATH = "C:/Users/JacobBrotmanKrass/Documents/Test_Parts_Library.xlsx"
VENDORS_SHEET_PATH = "C:/Users/JacobBrotmanKrass/Documents/Test_Vendor_Stock.xlsx"



editor = KicadDB(SYMBOLS_PATH, PARTS_SHEET_PATH)

editor.check_kicad_for_duplicate_BRIDs()
editor.check_database_for_duplicate_BRIDs()
editor.add_new_symbols_from_kicad()

sync_status = editor.kicad_parts_df.sort_values(by='BR ID', ignore_index=True).equals(editor.database_parts_df.sort_values(by='BR ID', ignore_index=True))

if sync_status == False:
    print("Differences detected between database and Kicad library")
    #editor.check_for_changes()

# At this stage, we've confirmed that all newly added parts from the Kicad library have been assigned BR numbers added to the database
# we have a few questions:
# 1. Are there any rows/parts that are in the database but not the Kicad library? Is this the place to do deal with pushing from database to Kicad?
# 2. Are there any existing cells that need updating? 
# Edge Case A: part is added in Kicad and given a unique BR ID (not generated in database) -- maybe this should just be heavily discouraged
# Edge Case B: part is deleted in Kicad library OR database
# All of this needs to be communicated to the Vendors database
# The thing is, we can only use pd.compare for 






Loading in libraries from Kicad...
No BR ID duplicates found in Kicad library!
No BR ID duplicates found in sheet!
C_0201_125n7F_25V_X7R_10%_1 assigned to BRE-000433
New Kicad part BRE-000433 added to database
C_0201_126n7F_25V_X7R_10%_1_1 assigned to BRE-000434
New Kicad part BRE-000434 added to database
C_0201_127n7F_25V_X7R_10%_1_1 assigned to BRE-000435
New Kicad part BRE-000435 added to database
C_0201_128n7F_25V_X7R_10%_1_1_1 assigned to BRE-000436
New Kicad part BRE-000436 added to database
Differences detected between database and Kicad library


In [65]:
editor.deal_with_extra_parts()

KeyError: "None of [Index([('BRE-000004',)], dtype='object', name='BR ID')] are in the [index]"

In [59]:
editor.database_parts_df

,Name,Description,Value,Symbol,Footprint,Datasheet,Manufacturer,MPN,Category
BR ID,,,,,,,,,
BRE-000000,C_0201_100nF_25V_X7R_10%,100nF ±10% 25V X7R 0201 Ceramic Capacitor,100nF 25V,BR_Capacitors_0201_:C_0201_100nF_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM033R61E104KE14J,Capacitors_0201_
BRE-000003,C_0201_1uF_16V_X5R_10%,1uF ±10% 16V X5R 0201 Ceramic Capacitor,1uF 16V,BR_Capacitors_0201_:C_0201_1uF_16V_X5R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.mouser.com/datasheet/2/281/1/GRM03...,Murata Electronics,GRM033R61C105ME15D,Capacitors_0201_
BRE-000005,C_0201_3p9F_50V_C0G_5%,3p9F ±5% 50V C0G 0201 Ceramic Capacitor,3p9F 50V,BR_Capacitors_0201_:C_0201_3p9F_50V_C0G_5%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM0335C1H3R9CA01D,Capacitors_0201_
BRE-000006,C_0201_470nF_6V3_X5R_10%,470nF ±10% 6V3 X5R 0201 Ceramic Capacitor,470nF 6V3,BR_Capacitors_0201_:C_0201_470nF_6V3_X5R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM033R60J474KE90D,Capacitors_0201_
BRE-000007,C_0201_470pF_50V_C0G_5%,470pF ±5% 50V C0G 0201 Ceramic Capacitor,470pF 50V,BR_Capacitors_0201_:C_0201_470pF_50V_C0G_5%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM0335C1H471JE01D,Capacitors_0201_
...,...,...,...,...,...,...,...,...,...
BRE-000432,SiR500DP,N-Channel 30 V (D-S) 150 °C MOSFET,SiR500DP,BR_Transistors:SiR500DP,BR_Discretes:Mosfet_Vishay_SiR500DP,https://www.vishay.com/docs/66840/sir500dp.pdf,Vishay Siliconix,SIR500DP-T1-RE3,Transistors
BRE-000433,C_0201_125n7F_25V_X7R_10%_1,125n7F ±10% 25V X7R 0201 Ceramic Capacitor,125n7F 25V,BR_Capacitors_0201_:C_0201_125n7F_25V_X7R_10%_1,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,M1111,Capacitors_0201_
BRE-000434,C_0201_126n7F_25V_X7R_10%_1_1,125n7F ±10% 25V X7R 0201 Ceramic Capacitor,126n7F 25V,BR_Capacitors_0201_:C_0201_126n7F_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,M2111,Capacitors_0201_


In [29]:

old_kc = old_kc.set_index("BR ID").sort_index()

# Figure out if there are any outlier parts that are only accounted for in one place
#kicad_extra_BRIDs = old_kc.index.difference(old_db).to_list()

In [17]:
old_db = editor.database_parts_df.copy()
old_db

,BR ID,Name,Description,Value,Symbol,Footprint,Datasheet,Manufacturer,MPN,Category
0,BRE-000000,C_0201_100nF_25V_X7R_10%,100nF ±10% 25V X7R 0201 Ceramic Capacitor,100nF 25V,BR_Capacitors_0201_:C_0201_100nF_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM033R61E104KE14J,Capacitors_0201_
1,BRE-000001,C_0201_10nF_25V_X7R_10%,10nF ±10% 25V X7R 0201 Ceramic Capacitor,10nF 25V,BR_Capacitors_0201_:C_0201_10nF_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://search.murata.co.jp/Ceramy/image/img/A...,Murata Electronics,GRM033R71E103KE14D,Capacitors_0201_
2,BRE-000003,C_0201_1uF_16V_X5R_10%,1uF ±10% 16V X5R 0201 Ceramic Capacitor,1uF 16V,BR_Capacitors_0201_:C_0201_1uF_16V_X5R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.mouser.com/datasheet/2/281/1/GRM03...,Murata Electronics,GRM033R61C105ME15D,Capacitors_0201_
3,BRE-000004,C_0201_220nF_25V_X5R_10%,220nF ±10% 25V X5R 0201 Ceramic Capacitor,220nF 25V,BR_Capacitors_0201_:C_0201_220nF_25V_X5R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM033R61E224KE01D,Capacitors_0201_
4,BRE-000005,C_0201_3p9F_50V_C0G_5%,3p9F ±5% 50V C0G 0201 Ceramic Capacitor,3p9F 50V,BR_Capacitors_0201_:C_0201_3p9F_50V_C0G_5%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM0335C1H3R9CA01D,Capacitors_0201_
...,...,...,...,...,...,...,...,...,...,...
431,BRE-000432,SiR500DP,N-Channel 30 V (D-S) 150 °C MOSFET,SiR500DP,BR_Transistors:SiR500DP,BR_Discretes:Mosfet_Vishay_SiR500DP,https://www.vishay.com/docs/66840/sir500dp.pdf,Vishay Siliconix,SIR500DP-T1-RE3,Transistors
432,BRE-000433,C_0201_125n7F_25V_X7R_10%_1,125n7F ±10% 25V X7R 0201 Ceramic Capacitor,125n7F 25V,BR_Capacitors_0201_:C_0201_125n7F_25V_X7R_10%_1,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,M1111,Capacitors_0201_
433,BRE-000434,C_0201_126n7F_25V_X7R_10%_1_1,125n7F ±10% 25V X7R 0201 Ceramic Capacitor,126n7F 25V,BR_Capacitors_0201_:C_0201_126n7F_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,M2111,Capacitors_0201_
434,BRE-000435,C_0201_127n7F_25V_X7R_10%_1_1,127n7F ±10% 25V X7R 0201 Ceramic Capacitor,127n7F 25V,BR_Capacitors_0201_:C_0201_127n7F_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,M1117,Capacitors_0201_


In [36]:
old_kc = editor.kicad_parts_df.copy()
old_kc

,BR ID,Name,Description,Value,Symbol,Footprint,Datasheet,Manufacturer,MPN,Category
0,BRE-000000,C_0201_100nF_25V_X7R_10%,100nF ±10% 25V X7R 0201 Ceramic Capacitor,100nF 25V,BR_Capacitors_0201_:C_0201_100nF_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM033R61E104KE14J,Capacitors_0201_
1,BRE-000433,C_0201_125n7F_25V_X7R_10%_1,125n7F ±10% 25V X7R 0201 Ceramic Capacitor,125n7F 25V,BR_Capacitors_0201_:C_0201_125n7F_25V_X7R_10%_1,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,M1111,Capacitors_0201_
2,BRE-000434,C_0201_126n7F_25V_X7R_10%_1_1,125n7F ±10% 25V X7R 0201 Ceramic Capacitor,126n7F 25V,BR_Capacitors_0201_:C_0201_126n7F_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,M2111,Capacitors_0201_
3,BRE-000435,C_0201_127n7F_25V_X7R_10%_1_1,127n7F ±10% 25V X7R 0201 Ceramic Capacitor,127n7F 25V,BR_Capacitors_0201_:C_0201_127n7F_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,M1117,Capacitors_0201_
4,BRE-000436,C_0201_128n7F_25V_X7R_10%_1_1_1,128n7F ±10% 25V X7R 0201 Ceramic Capacitor,128n7F 25V,BR_Capacitors_0201_:C_0201_128n7F_25V_X7R_10%_1,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,M2119,Capacitors_0201_
...,...,...,...,...,...,...,...,...,...,...
430,BRE-000428,NTGS5120PT1G,MOSFET P-CH 60V 1.8A 6TSOP,NTGS5120PT1G,BR_Transistors:NTGS5120PT1G,BR_SOP:TSOP-6_ON_SEMI,https://www.onsemi.com/pdf/datasheet/ntgs5120p...,onsemi,NTGS5120PT1G,Transistors
431,BRE-000429,NXV55UNR,"N-Channel 30 V 1.9A (Ta) 340mW (Ta), 2.1W (Tc)...",NXV55UNR,BR_Transistors:NXV55UNR,BR_SOT:SOT-23,https://assets.nexperia.com/documents/data-she...,Nexperia USA Inc.,NXV55UNR,Transistors
432,BRE-000430,SI2305CDS-T1-GE3,Mosfet P-ch 8V 5.8A SOT23-3,SI2305CDS-T1-GE3,BR_Transistors:SI2305CDS-T1-GE3,Package_TO_SOT_SMD:SOT-23,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Vishay,SI2305CDS-T1-GE3,Transistors
433,BRE-000431,SQJ974EP,Mosfet Array 100V 30A (Tc) 48W Surface Mount P...,SQJ974EP,BR_Transistors:SQJ974EP,BR_SOP:PowerPAK_SO-8_Dual,https://www.vishay.com/docs/78092/sqj974ep.pdf,Vishay Siliconix,SQJ974EP-T1_GE3,Transistors


In [86]:
testDB = editor.database_parts_df.set_index("BR ID").sort_index()
testKC = editor.kicad_parts_df.set_index("BR ID").sort_index()
testDB.compare(testKC, result_names=("db", "kc"))

Manufacturer                      MPN  \
                            db   kc                  db   
BR ID                                                     
BRE-000004                 NaN  NaN  GRM033R61E224KE01D   
BRE-000007  Murata Electronics                      NaN   

                                        
                                    kc  
BR ID                                   
BRE-000004  GRM033R61E224KE01Dasdfasdf  
BRE-000007                         NaN

In [61]:


### CHECKING FOR UPDATES MUST BE DONE ON SAME SIZED DATAFRAMES WITH THE SAME INDICES (BR NUMBERS)

# Compares the entries from Kicad and the database
changes = editor.kicad_parts_df.compare(editor.database_parts_df, result_names=("Kicad", "Database"))
for idx, row in changes.iterrows():

    print(f"The following differences were detected between Kicad library and the database for {editor.kicad_parts_df.loc[idx, 'BR ID']}")
    
    # Isolate which cells were changed and print to user
    changed_cells = row[~row.isna()]
    print(changed_cells)

# Update database with any cell that holds a different value in the Kicad library
database_parts_df.update(kicad_parts_df)




um


Manufacturer                                             MPN  \
         Kicad            Database                       Kicad   
4          NaN                 NaN  GRM033R61E224KE01Dasdfasdf   
7               Murata Electronics                         NaN   

                       
             Database  
4  GRM033R61E224KE01D  
7                 NaN

In [72]:
for idx, row in changes.iterrows():
    print(f"The following differences were detected between Kicad library and the database for {editor.kicad_parts_df.loc[idx, 'BR ID']}")
    changed_cells = row[~row.isna()]
    print(changed_cells)
while(1):
    push_confirmation = input("Would you like to push from Kicad to the database? [y/n]")
    if push_confirmation == 'y':
        editor.kicad_parts_df = kicad_parts_df.copy()
        editor.database_parts_df = database_parts_df.copy()
        print(f"{editor.database_parts_df.loc[changes.index, 'BR ID'].to_list()} were updated in the database.")
        break
    elif push_confirmation == 'n':
        print("Changes not pushed")
        break
    else:
        print("invalid entry")



The following differences were detected between Kicad library and the database for BRE-000004
MPN  Kicad       GRM033R61E224KE01Dasdfasdf
     Database            GRM033R61E224KE01D
Name: 4, dtype: object
The following differences were detected between Kicad library and the database for BRE-000007
Manufacturer  Kicad                         
              Database    Murata Electronics
Name: 7, dtype: object
['BRE-000004', 'BRE-000007'] were updated in the database.


In [202]:
testKC.sort_values(by='BR ID', ignore_index=True).iloc[72:74]

,BR ID,Name,Description,Value,Symbol,Footprint,Datasheet,Manufacturer,MPN,Category
72,BRE-000072,Conn_01x04,,Conn_01x04,BR_Connectors:Conn_01x04,,,,,Connectors
73,BRE-000073,Conn_01x08,Connector Header Through Hole 8 position 0.100...,PH1-08-UA,BR_Connectors:Conn_01x08,BR_Connectors:PinHeader_1x08_P2.54mm_Vertical,https://app.adam-tech.com/products/download/da...,Adam Tech,PH1-08-UA,Connectors


In [203]:
testDB.sort_values(by='BR ID', ignore_index=True).iloc[72:74]


,BR ID,Name,Description,Value,Symbol,Footprint,Datasheet,Manufacturer,MPN,Category
72,BRE-000072,Conn_01x04,,Conn_01x04,BR_Connectors:Conn_01x04,,,,,Connectors
73,BRE-000073,Conn_01x08,Connector Header Through Hole 8 position 0.100...,PH1-08-UA,BR_Connectors:Conn_01x08,BR_Connectors:PinHeader_1x08_P2.54mm_Vertical,https://app.adam-tech.com/products/download/da...,Adam Tech,PH1-08-UA,Connectors


In [204]:
pd.testing.assert_frame_equal(testKC.sort_values(by='BR ID', ignore_index=True), testDB.sort_values(by='BR ID', ignore_index=True))

AssertionError: DataFrame.iloc[:, 2] (column name="Description") are different

DataFrame.iloc[:, 2] (column name="Description") values are different (1.6092 %)
[index]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
[left]:  [100nF ±10% 25V X7R 0201 Ceramic Capacitor, 10nF ±10% 25V X7R 0201 Ceramic Capacitor, 1nF ±1% 50V C0G 0201 Ceramic Capacitor, 1uF ±10% 16V X5R 0201 Ceramic Capacitor, 220nF ±10% 25V X5R 0201 Ceramic Capacitor, 3p9F ±5% 50V C0G 0201 Ceramic Capacitor, 470nF ±10% 6V3 X5R 0201 Ceramic Capacitor, 470pF ±5% 50V C0G 0201 Ceramic Capacitor, 4n7F ±10% 25V X7R 0201 Ceramic Capacitor, 6pF ±15% 50V C0G 0201 Ceramic Capacitor, 0.1 µF